In [1]:
import pandas as pd
import webbrowser
import os


In [2]:
df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vR-BAUvNUjp2AeV_daeeqHReX0M3ew3ZpEL3nfkrz96uUd816mV_hV1uWMvbsACphEBGjqHJBswGwFz/pub?gid=614465369&single=true&output=csv")

In [3]:
# filter dataframe depending on the value of two columns
df = df[(df['Curso Escolar'] == "2022-23") & (df['Convocatoria'] == 1)]

In [4]:
# open the first row in a new tab
def open_url(row):
    course_id = row["Moodle_url"].split("id=")[-1]
    url_course_completion = f"https://moodle.catedu.es/course/completion.php?id={course_id}"
    url_activity_completion = f"https://moodle.catedu.es/course/bulkcompletion.php?id={course_id}"
    webbrowser.open_new_tab(url_course_completion)
    webbrowser.open_new_tab(url_activity_completion)


In [5]:
# apply the function to the dataframe
_ = df[130:].apply(open_url, axis=1) # hecho hasta el 130

In [9]:
len(df)

140

In [18]:
df["Tutorizado"]

0      False
1      False
2      False
3      False
4      False
       ...  
171    False
172    False
173    False
174    False
175    False
Name: Tutorizado, Length: 140, dtype: object